In [101]:
import iris
import os
import glob as sir_globington_the_file_gatherer
import datetime as datetime
import iris.coord_categorisation as cat
import sys
import time
import multiprocessing as mp

# Set up path to root directory
root_fp = "/nfs/a319/gy17m2a/PhD/"
os.chdir(root_fp)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Create path to files containing functions
sys.path.insert(0, root_fp + 'Scripts/GlobalFunctions')
from Spatial_plotting_functions import *
from Spatial_geometry_functions import *

##################################################################
# This is a square area surrounding Leeds
leeds_at_centre_gdf = create_leeds_at_centre_outline({'init' :'epsg:3857'})

# ### Establish the ensemble member
trim_to_leeds = False

ems =['04', '06', '07', '08', '09', '10', '11', '12', '13', '15']
em_matching_dict = {'01':'bc005', '04': 'bc006', '05': 'bc007', '06':'bc009',  '07':'bc010', 
                    '08': 'bc011', '09':'bc013', '10': 'bc015', '11': 'bc016', '12': 'bc017', '13':'bc018', '15':'bc012'}
ems = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc013',  'bc015',  'bc016', 'bc017', 'bc018', 'bc012']

In [106]:
yrs_range = "2001_2020"
resolution = '2.2km' #2.2km, 12km, 2.2km_regridded_12km

for em in ems:

    ddir = f"ProcessedData/TimeSeries/UKCP18_every30mins/{resolution}/{yrs_range}/leeds-at-centre/{em}/"
    ### Save as numpy array
    #print("saving data")
    if not os.path.isdir(ddir):
        os.makedirs(ddir)

    print(em, resolution)

    # ### Get a list of filenames for this ensemble member, for just JJA
    if resolution == '2.2km':
        general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/{resolution}/{em}/{yrs_range}/{em}a.pr*'
    elif resolution == '12km':
          general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/{resolution}/{em}/{yrs_range}/pr_rcp85_land-rcm_uk_12km_{em}_day_*'
    elif resolution == '2.2km_regridded_12km':
        general_filename = f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/{resolution}/{em}/NearestNeighbour/{yrs_range}/rg_pr_rcp85_land-cpm_uk_2.2km_{em}_1hr_*'
    general_filename

    filenames = []
    for filename in sir_globington_the_file_gatherer.glob(general_filename):
        if resolution == '2.2km':
            if filename[84:86] in ['06', '07', '08']:
                filenames.append(filename)
    #     elif resolution == '12km':
    #         if filename[95:99] in ['2000', '1990', '1980']:
    #             filenames.append(filename)
    #     elif resolution == '2.2km_regridded_12km':
    #         if filename[132:136] in ['2000', '1990', '1980']:
    #             filenames.append(filename)      
    # print(len(filenames))

    # ### Load in the data
    monthly_cubes_list = iris.load(filenames)

    ### Concatenate cubes into one
    model_cube_jja = monthly_cubes_list.concatenate_cube()      

    ### Get associated times
    times = model_cube_jja.coord('time')   
    # print(times.units.num2date(times.points))
    np.save(f"ProcessedData/TimeSeries/UKCP18_every30mins/{resolution}/{yrs_range}/leeds-at-centre/timestamps.npy", times) 

    ################################################################
    # Create a numpy array containing all the precipitation values from across
    # all 20 years of data and all positions in the cube
    ################################################################
    # Define length of variables defining spatial positions
    lat_length= model_cube_jja.shape[1]
    lon_length= model_cube_jja.shape[2]
    #print("Defined length of coordinate dimensions")
    #print(lat_length, lon_length)        

    # # # Load data
    #print("Loading data")
    data = model_cube_jja.data
    #print("Loaded data")

    start = time.time()

    # Create an empty array to fill with data
    all_the_data = np.array([])

    #print("Entering loop through coordinates")
    total = 0
    for i in range(0,lat_length):
        for j in range(0,lon_length):
            # Print the position
            if i in [10,20,30] and j ==0:
                print(i,j)     
            data_slice = data[:,i,j]
            # Remove mask
            data_slice = data_slice.data
            # Add to total
            total = total + data_slice.shape[0]

            # Add the slice to the array containing all the data from all the locations
            all_the_data = np.append(all_the_data,data_slice)

    ### Save as numpy array
    #print("saving data")
    if not os.path.isdir(ddir):
        os.makedirs(ddir)

    np.save(ddir + "leeds-at-centre_jja.npy", all_the_data)   

    end= time.time()
    print(f"Time taken is {(end-start)/60}seconds" )

bc005 2.2km
10 0
20 0
30 0
Time taken is 6.489204609394074seconds
bc006 2.2km
10 0
20 0
30 0
Time taken is 6.8136972745259605seconds
bc007 2.2km
10 0
20 0
30 0
Time taken is 6.687424929936727seconds
bc009 2.2km
10 0
20 0
30 0
Time taken is 6.6749387303988135seconds
bc010 2.2km
10 0
20 0
30 0
Time taken is 6.54090967575709seconds
bc011 2.2km
10 0
20 0
30 0
Time taken is 6.612638254960378seconds
bc013 2.2km
10 0
20 0
30 0
Time taken is 6.635568447907766seconds
bc015 2.2km
10 0
20 0
30 0
Time taken is 6.659102241198222seconds
bc016 2.2km
10 0
20 0
30 0
Time taken is 6.766812137762705seconds
bc017 2.2km
10 0
20 0
30 0
Time taken is 6.57525186141332seconds
bc018 2.2km
10 0
20 0
30 0
Time taken is 3.7688184142112733seconds
bc012 2.2km
10 0
20 0
30 0
Time taken is 0.8989796916643779seconds
